In [25]:
import llm_core.llm as L
from srs.SRS import SRS
import re
import json

In [26]:
#initialize
llm1 = L.LLM("You are a grader for beginner language learning students. "
            "You judge their response for grammar and "
            "concisely describe how each word is correct or incorrect in the sentence.")
llm2 = L.LLM("""You are a grader for beginner language learning students.
                You judge their response for grammar and correctness and grade accordingly.
                Each mistake has a short description with it for what they did wrong.
                If there is nothing wrong, simply don't say anything.""")
srs = SRS()


In [27]:
#mistakes in first and last sentence
response = "Io vado a la scuola. Oggi è una bella giornata. Lei mangia molto di pizza."

# simply grade each word in the context of the sentence
def grade_response(response : str) -> list:
    output = list()
    sentences = [sentence for sentence in response.split('.') if sentence != '']
    for sentence in sentences:
        words = sentence.split(' ')
        for word in words:
            llm1.reset()
            grade1 = llm1(
                f"Evaluate {word} for grammatical correctness in the sentence: {sentence}",
                response_format={
                    'reason': str, 'Correct? yes or no?' : str
                },
                max_tokens=1000,
                temperature=0,
                verbose=False
            )
            output.append(grade1)
    return output

# remind system with what it's already graded in the sentence
def grade_response2(response : str) -> list:
    output = list()
    sentences = [sentence for sentence in response.split('.') if sentence != '']
    for sentence in sentences:
        grades = ''
        words = sentence.split(' ')
        for word in words:
            llm1.reset()
            grade1 = llm1(
                grades_prompt(grades) + 
                f"Evaluate {word} for grammatical correctness in the sentence: {sentence}",
                response_format={
                    'reason': str, 'Correct? yes or no?' : str
                },
                max_tokens=None,
                temperature=0,
                verbose=False
            )
            output.append(grade1)
            grades = str(output)
    return output

def grades_prompt(grades : str) -> str:
    if (grades != ''):
        grades = "You have evaluated the sentence so far as follows: " + grades
    return grades

#print(json.dumps(grade_response(response), indent=4))
#print('####################################')
#print(json.dumps(grade_response2(response), indent=4))

In [28]:
finnish_words = [ #initial words
    "hei",  # hello
    "moi",  # hi
    "kiitos",  # thank you
    "kiitos paljon",  # thank you very much
    "anteeksi",  # excuse me
    "vettä",  # water
    "kahvia",  # coffee
    "ruokaa",  # food
    "keskustelu",  # conversation
    "hauska",  # fun/nice
    "hyvin",  # well
    "huonosti",  # poorly
    "mikä",  # what
    "missä",  # where
    "kuinka",  # how
    "miksi",  # why
    "kuka",  # who
    "tämä",  # this
    "tuo",  # that
    "joo",  # yes
    "ei",  # no
    "kiitos ei",  # no thank you
    "mutta",  # but
    "ja",  # and
    "tai",  # or
    "nyt",  # now
    "tulee",  # comes
    "mennä",  # go
    "tullaan",  # we come
    "paljon",  # a lot/much
    "vähän",  # little
    "hyvä",  # good
    "huono",  # bad
    "kiva",  # nice
    "suosikki",  # favorite
    "koulutus",  # education
    "työ",  # work
    "raha",  # money
    "perhe",  # family
    "ystävä",  # friend
    "rakastaa",  # love
    "talo",  # house
    "auto",  # car
    "puhelin",  # phone
    "kirja",  # book
    "lehti",  # newspaper
    "kauppa",  # shop
    "ravintola",  # restaurant
    "kaupungissa",  # in the city
    "metsä",  # forest
    "järvi",  # lake
    "meri",  # sea
    "taivas",  # sky
    "aurinko",  # sun
    "kuu",  # moon
    "tähti",  # star
    "sateenkaarre",  # rainbow
    "kello",  # clock
    "aika",  # time
    "päivä",  # day
    "yö",  # night
    "viikonpäivä",  # weekday
    "maanantai",  # Monday
    "tiistai",  # Tuesday
    "keskiviikko",  # Wednesday
    "torstai",  # Thursday
    "perjantai",  # Friday
    "lauantai",  # Saturday
    "sunnuntai",  # Sunday
    "tammikuu",  # January
    "helmikuu",  # February
    "maaliskuu",  # March
    "huhtikuu",  # April
    "toukokuu",  # May
    "kesäkuu",  # June
    "heinäkuu",  # July
    "elokuu",  # August
    "syyskuu",  # September
    "lokakuu",  # October
    "marraskuu",  # November
    "joulukuu",  # December
    "kesä",  # summer
    "syksy",  # autumn
    "talvi",  # winter
    "kevät",  # spring
    "lämpö",  # heat
    "kylmä",  # cold
    "sateinen",  # rainy
    "tuulinen",  # windy
    "kaunis",  # beautiful
    "rumi",  # ugly
    "pitkä",  # tall
    "lyhyt",  # short
    "iso",  # big
    "pieni",  # small
    "nopea",  # fast
    "hidas",  # slow
    "puhtaus",  # cleanliness
    "likainen",  # dirty
    "terve",  # healthy
    "sairas",  # sick
]


In [29]:
TEACHER_NAME = 'Rose' # localized to target language (regular 'e' for Finnish)
LEARNER_NAME = 'Lucas' # obtained from user profile

allowed_vocab = [
    # greetings
    'terve', 'hei',

    # nouns
    'talo',     # house
    'vesi',     # water
    'ystävä',   # friend
    'huomenta', # morning
    'velho',    # wizard
    'suomi',    # Finland
    'koira',    # dog
    'nimi',     # name

    # singular possessive nouns for 'nimi'
    'nimeni',   # first person "my name"
    'nimesi',   # second person "your name"
    'nimensä',  # third person "his name"

    # singular posessive nous for 'ystävä'
    'ystäväni',  # first person
    'ystäväsi',  # second person
    'ystävänsä', # third person

    # adjectives
    'vanha',       # old
    'hyvää',       # good
    'suomalainen', # Finnish
    'mukava',      # nice

    # pronouns, posesives, "to be" verbs
    'minä', 'minun', 'olen', 'olenko', # first person
    'sinä', 'sinun', 'olet', 'oletko', # second person
    'hän', 'hänen', 'on', 'onko',      # third person

    # names
    'matti', 'aleksi', 'sami', TEACHER_NAME.lower(), LEARNER_NAME.lower(),

    # useful words
    'kyllä', # yes
    'ei', # no
    'mitä', # "what/how" as in "what did you say?" or "how are you" -- about more abstract things
    'mikä', # "what" as in "what is this?" or "what is your name?" -- about specific things
]

finnish_words = ""
for i in range(len(allowed_vocab)):
    finnish_words = finnish_words + allowed_vocab[i]

In [30]:
# Question-asking
question_llm = L.LLM("You are a helpful language learning assistant.")
def get_question(question_llm, words, language, n):
    question = question_llm(
        f"Select {n} words from the following list and write a simple sentence using them in {language}. "
        "Then, write a simple question about the sentence to see if the user understands, in English. "
        "Format as {'sentence':<sentence>, 'question':<question>, 'answer':<answer>}."
        f"List of words: {words}",
        response_format={'Sentence':str, 'Question':str, 'Answer':str},
        max_tokens=None,
        temperature=0,
        verbose=False
    )
    return question

print(get_question(question_llm, finnish_words, 'Finnish', 3))

{'Sentence': 'Olen suomalaista.', 'Question': 'What does this sentence mean in English?', 'Answer': 'I am Finnish.'}


In [ ]:
question_llm.reset()
n = 2
language = "Finnish"
sentence_llm = L.LLM(f"You are a helpful language learning assistant. You respond using {n} words from your dictionary "
                     "to create simple sentences in {language}. ")
question_llm = L.LLM(f"You are a helpful language learning assistant. You ask users questions about sentences in {language} "
                     "to test if they understand the meaning. Use English to ask questions")
def get_sentence(question_llm, n, words, language):
    sentence = question_llm(
        f"Select {n} words from the following list and write a simple sentence using them in {language}. "
        "Then, write a simple question about the sentence to see if the user understands, in English. "
        f"List of words: {words}",
        response_format={'Sentence':str},
        max_tokens=None,
        temperature=0,
        verbose=False
    )
    return sentence

def get_question2(question_llm, input_sentence, language):
    question = question_llm(
        f"Write an intelligent question in English about a word in the following sentence to see if "
        "the user understands, followed by an answer: {input_sentence}",
        response_format={'Question':str, 'Answer':str},
        max_tokens=None,
        temperature=0.1,
        verbose=False
    )
    return question

def test(sentence_llm, question_llm, language):
    sentence = get_sentence(sentence_llm, 3, finnish_words, language)
    print(sentence)
    question = get_question2(question_llm, sentence, language)
    print(question)
    return (sentence, question)

for i in range(5):
    test(sentence_llm, question_llm, 'Finnish')

{'Sentence': 'Olen suomalaista.'}
{'Question': "What does 'suomalaista' mean in the sentence 'Olen suomalaista.'?", 'Answer': "The word 'suomalaista' means 'Finnish' in this sentence."}
{'Sentence': 'Lovesi hyvää.'}
{'Question': "What does 'hyvää' mean in the sentence 'Lovesi hyvää.'?", 'Answer': "The word 'hyvää' means 'good' in this sentence."}
{'Sentence': 'Olen suomalainen.'}
{'Question': "What does 'suomalainen' mean in the sentence 'Olen suomalainen.'?", 'Answer': "The word 'suomalainen' means 'Finnish' in this sentence."}
{'Sentence': 'Lovesi vanhää.'}
{'Question': "What does 'vanhää' mean in the sentence 'Lovesi vanhää.'?", 'Answer': "The word 'vanhää' means 'old' in this sentence."}
{'Sentence': 'Olen suomalaista.'}
{'Question': "What does 'suomalaista' mean in the sentence 'Olen suomalaista.'?", 'Answer': "The word 'suomalaista' means 'Finnish' in this sentence."}


In [ ]:
# question judger
eval_llm = L.LLM("You are a helpful language learning assistant. "
                 f"You are given questions and answers to sentences in {language} and determine if they are correct.")
def evaluate(eval_llm, sentence, question, correct_answer, user_answer):
    evaluation = eval_llm(
        f"Is this answer correct? Sentence: {sentence}, question: {question}, correct answer: {correct_answer}, user answer: {user_answer}",
        response_format={'Correct (yes or no)':str, 'Reasoning':str},
        max_tokens=None,
        temperature=0.1,
        verbose=False
    )
    return evaluation

t = test(sentence_llm, question_llm, 'Finnish')
print(evaluate(eval_llm, t[0]['Sentence'], t[1]['Question'], t[1]['Answer'], 'hauska sinua'))

{'Sentence': 'Olen mukavissa.'}
{'Question': "What does 'mukavissa' mean in the sentence 'Olen mukavissa.'?", 'Answer': "The word 'mukavissa' means 'comfortable' in this sentence."}
{'Correct (yes or no)': 'no', 'Reasoning': "The user answer 'hauska sinua' does not match the correct answer 'The word 'mukavissa' means 'comfortable' in this sentence.'"}


In [1]:
from llm_core import learning_with_variation
learning_with_variation.main()

Initializing Global LLM Instance
Tämä on ollut minun koirani.
What was Mine?
{'Answer': 'dog'}
The answer "dog" is correct. The sentence "Tämä on ollut minun koirani." translates to "This was my dog." in English, so identifying the correct answer as "dog" is accurate. However, the user provided the answer as "dog" without the quotes or punctuation that would properly format it as a text answer. Despite this formatting issue, the content of the answer is correct.
